# PROJECT

### Download of the NSP13 Protein (6zsl comformation)

In [7]:
import Bio
from Bio.PDB import PDBList
#from pathlib import Path

def retrive_nsp13(structure):
    """Download a NSP13 structure from the PDB
    Returns
    -------
    structure : .cif file of the structure
    """
    pdbl = PDBList()
    pdbl.download_pdb_files([structure], pdir="NSP13", file_format="pdb")
    #MEET_U --> meetu2024
    #fpath = Path.home() / f"Desktop/M2/S1/meetu2024/2023-2024_Team4-SU/NSP13/{structure}.pdb"
    fpath = f"./NSP13/{structure}.pdb"
    with open(fpath) as f:
        return f.read()
    
#structure="6zsl"
#nsp13 = retrive_nsp13(structure)
#nsp13

### VISUALIZATION WITH PYMOL

In [8]:
import pymol
import cmd 

#Visualization w Pymol

def visualize_protein(pdb_code):
    """
    Visualize a protein structure using PyMOL.

    Parameters:
    - pdb_code (str): The PDB code of the protein.

    Example:
    visualize_protein("6zsl")
    """
    # Start PyMOL
    pymol.finish_launching(['pymol', '-qc'])

    # Fetch the protein structure
    cmd.fetch(pdb_code)

    # Show the protein as a ribbon
    cmd.show("cartoon")

    # Display the protein structure
    cmd.zoom()

    # Keep the PyMOL window open
    cmd.rock()

#visualize_protein(nsp13)

PyMOL failed to start, dumping diagnostics...
pymol-2.5.7-py312h1880af8_0.json
conda build: py312h1880af8_0 https://conda.anaconda.org/schrodinger/win-64
pymol-bundle-2.5.7-0.json
conda build: 0 https://conda.anaconda.org/schrodinger/win-64
pymol-upgrade-helper-1.1-1.json
conda build: 1 https://conda.anaconda.org/schrodinger/win-64
pymol-web-examples-2.4-1.json
conda build: 1 https://conda.anaconda.org/schrodinger/noarch

Operating System:
Windows-11-10.0.22621-SP0
PyQt5 5.15.10 (Qt 5.15.2)

Python:
3.12.1 | packaged by conda-forge | (main, Dec 23 2023, 07:53:56) [MSC v.1937 64 bit (AMD64)]
prefix=c:\Users\LALIN\anaconda3\envs\pymol
executable=c:\Users\LALIN\anaconda3\envs\pymol\python.exe
filesystemencoding=utf-8

Qt, Python and PyMOL Environment Variables:
PYDEVD_IPYTHON_COMPATIBLE_DEBUGGING=1
PYDEVD_USE_FRAME_EVAL=NO
PYMOL_DATA=c:\Users\LALIN\anaconda3\envs\pymol\share\pymol\data
PYMOL_HOME=C:\Users\LALIN\AppData\Local\Schrodinger\PyMOL2
PYMOL_PATH=c:\Users\LALIN\anaconda3\envs\pymo

ImportError: DLL load failed while importing _cmd: Le module spécifié est introuvable.

### POCKET DETECTION (P2RANK)

1. Pocket Generation

In [ ]:
#Pocket creation script 

import subprocess

# Path to the p2rank script (prank.sh)
p2rank_path = './p2rank/prank.sh'

# Path to Git Bash for the Windows environment (to be replaced or improved if necessary)
bash_path = 'c:/PROGRA~1/Git/bin/bash.exe'  # Make sure to set the correct path

# Folder to store pocket prediction results
output_folder = './pockets'

# Command to execute p2rank and predict pockets
command = f'{bash_path} {p2rank_path} predict -f ./NSP13/6zsl.cif -o {output_folder}'

# Execution of the command
subprocess.run(command, shell=True)


CompletedProcess(args='c:/PROGRA~1/Git/bin/bash.exe ./p2rank/prank.sh predict -f ./NSP13/6zsl.cif -o ./pockets', returncode=1)

2. Display of the pockets of the 6ZSL comformation

In [ ]:
#Reading / Display of the pockets

import pandas as pd

csv = f'./pockets/6zsl.cif_predictions.csv'
r_csv = pd.read_csv(csv)

df = pd.DataFrame(r_csv)
print(df.head(2))

   name         rank     score   probability   sas_points   surf_atoms  \
0  pocket1         1     16.76         0.786           95           54   
1  pocket2         2     12.94         0.668           84           38   

      center_x     center_y     center_z  \
0     -13.6826      11.0843     -74.6564   
1     -18.4473      34.5627     -22.8999   

                                         residue_ids  \
0   A_284 A_285 A_287 A_288 A_289 A_290 A_312 A_3...   
1   B_284 B_285 B_287 B_288 B_289 B_313 B_316 B_3...   

                                       surf_atom_ids  
0   6511 6517 6518 6519 6520 6528 6529 6539 6540 ...  
1   2116 2122 2123 2133 2134 2137 2141 2144 2145 ...  


### LIGAND PREPARATION

1. Downloading the ligands from the ZINC database w the script

In [ ]:
import subprocess

file = f'./ligands-downloader/ligand_downloader.py'

# Exécution du script
process = subprocess.run(['python', file], capture_output=True, text=True)


2. Display of the downloaded ligands 

In [ ]:
import os

l_file = f'./ligands/zinc_database/'

#Command to read all the zinc files that were downloaded and put them in a dataframe 
ligands_rep = [lig for lig in os.listdir(l_file) if os.path.isfile(os.path.join(l_file, lig))]
pd_ligrep = pd.DataFrame(ligands_rep)

#Display of the first 3 (to make the file more readable)
print(pd_ligrep.head(3))
#for nom_dossier in noms_dossiers:
#    print(ligands_rep)

                      0
0  ZINC001485529916.sdf
1  ZINC001485529917.sdf
2  ZINC001485529920.sdf


3. Preparation of the ligands for the docking with Rdkit

    3.1 - Utilisation de RDKIT pour préparer les ligands (ajout d'hydrogènes, génération de comformation de molécules...)

In [ ]:
#Preparation avec rdkit

from rdkit import Chem
from rdkit.Chem import AllChem
#from tqdm import tqdm
import os

#Lien avec le fichier csv contenant la liste des ligands à télécharger
csv_file = './ligands/ligands_downloader/csv_files/substances.csv'  
df = pd.read_csv(csv_file,sep = ';')

# Charger les fichiers SDF et préparer les ligands qu'on a téléchargé
ligands_folder = './ligands/zinc_database/'
os.makedirs(ligands_folder, exist_ok=True)

for zinc_id in df['zinc_id']:
    sdf_file = f'{ligands_folder}/{zinc_id}.sdf'

    # Charger les molécules depuis le fichier SDF
    suppl = Chem.SDMolSupplier(sdf_file)
    ligands = [mol for mol in suppl if mol is not None]

    # Ajout des hydrogènes explicites et génération des conformations 3D
    for mol in ligands:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)

    # Sauvegarder les molécules modifiées

    os.makedirs('./ligands/zinc_db_prepared/', exist_ok=True)
    modified_sdf_file = f'./ligands/zinc_db_prepared/{zinc_id}_prepared.sdf'
    w = Chem.SDWriter(modified_sdf_file)
    for mol in ligands:
        w.write(mol)
    w.close()

    3.2 - Utilisation de openbabel pour convertir les fichiers .sdf préparés en .pdbqt

In [ ]:
#conversion en pdqtb avec openbabel

import os
import subprocess

# Chemin des fichiers SDF et dossier de sortie
ligands_folder = './ligands/zinc_db_prepared/'
output_folder = './ligands/zinc_db_pdbqt/'
os.makedirs(output_folder, exist_ok=True)

for zinc_id in df['zinc_id']:
    sdf_file = f'{ligands_folder}/{zinc_id}_prepared.sdf'
    output_pdbqt_file = f'{output_folder}/{zinc_id}_prepared.pdbqt'

    # Utilisation de Open Babel pour convertir SDF en PDBQT
    subprocess.run(['babel', sdf_file, '-O', output_pdbqt_file, '-xh'])  # -xh ajoute des hydrogènes si nécessaire

    #print(f"Fichier PDBQT généré pour {zinc_id}")


4. Display of the prepared ligands

In [ ]:
import os
import pandas as pd 

l_file = f'./ligands/zinc_db_pdbqt/'

#Command to read all the zinc files that were downloaded and put them in a dataframe 
pligpd_rep = [lig for lig in os.listdir(l_file) if os.path.isfile(os.path.join(l_file, lig))]
pd_ligpd = pd.DataFrame(pligpd_rep)

#Display of the first 3 (to make the file more readable)
print(pd_ligpd.head(3))
#for nom_dossier in noms_dossiers:
#    print(ligands_rep)

                                 0
0  ZINC001485529916_prepared.pdbqt
1  ZINC001485529917_prepared.pdbqt
2  ZINC001485529920_prepared.pdbqt


### DOCKING ENGINE

1. Selection of the binding pocket

In [9]:
import pandas as pd
# 1. Retrieve the pocket

pockets = pd.read_csv("pockets/6zsl.cif_predictions.csv")
pockets.columns = [col.strip() for col in pockets.columns]
best_pocket = pockets.query("rank == 1") 
(center_x, center_y, center_z) = best_pocket[["center_x", "center_y", "center_z"]].iloc[0]

#print(best_pocket)

2. Transformation of the pocket in .pdbqt w openbabel

In [23]:
import subprocess

def transformation_pdb_to_pdbqt(structure_name, pdb_content_lines):
    structure_pdb = f"./NSP13/{structure_name}.pdb"
    structure_pdbqt = f"./NSP13/{structure_name}.pdbqt"
    
     # Convertis la liste de lignes en une seule chaîne
    pdb_content = '\n'.join(pdb_content_lines)
    
    with open(structure_pdb, 'w') as f:
        f.write(pdb_content)
    
    subprocess.run(['babel', structure_pdb, '-O', structure_pdbqt])
    
    return structure_pdbqt


structure_name ="6zsl"
structure_content = retrive_nsp13(structure_name)
nsp13_pdbqt = transformation_pdb_to_pdbqt(structure_name, structure_content)


In [24]:
# 2. Retrieve the ligand and the protein in format .pdbqt
# Conversion was done manually using autodocks GUI interface, 
# with the following tutorial:
# https://www.youtube.com/watch?v=LqRy24lnHVM
# It can certainly be done by command line
fpath_ligand = "ligands/zinc_db_pdbqt/ZINC001485529916_prepared.pdbqt"
fpath_prot = "NSP13/6zsl.pdbqt" 

# 3. Create config file
config = f"""receptor = {fpath_prot}
ligand = {fpath_ligand}

center_x = {center_x}
center_y = {center_y}
center_z = {center_z}

size_x = 80
size_y = 80
size_z = 80

out = vina_out.pdbqt
log = logs_vina.txt
num_modes = 10
energy_range = 4"""

file_path = "config.txt"
with open(file_path, "w") as file:
    file.write(config)

In [25]:
import subprocess
vina_path = "../Meet-U-2023-2024/dockingjob/vina/vina.exe"
subprocess.run([vina_path, "--config", "config.txt"], capture_output=True, text=True)

CompletedProcess(args=['../Meet-U-2023-2024/dockingjob/vina/vina.exe', '--config', 'config.txt'], returncode=0, stdout='#################################################################\n# If you used AutoDock Vina in your work, please cite:          #\n#                                                               #\n# O. Trott, A. J. Olson,                                        #\n# AutoDock Vina: improving the speed and accuracy of docking    #\n# with a new scoring function, efficient optimization and       #\n# multithreading, Journal of Computational Chemistry 31 (2010)  #\n# 455-461                                                       #\n#                                                               #\n# DOI 10.1002/jcc.21334                                         #\n#                                                               #\n# Please see http://vina.scripps.edu for more information.      #\n#################################################################\n\nWARNING